In [1]:
import pyodbc
import pandas as pd
import datetime as dt
import numpy as np
from datetime import timedelta
from datetime import datetime
from dateutil import parser as dtparser

import pandasql as ps

def dt_form(x):
    return dt.date.isoformat(x)


#Code to connect to data warehouse. driver = type of server. 
# server = sql server's name and database = which database on the sql server you want to access.
# trusted connection = yes just means you won't have a security error thrown.
dw = pyodbc.connect(driver = '{SQL SERVER}', 
                            Server = 'acmdw',
                            Database = 'DW_ACM',
                            trusted_connection = 'yes')

In [174]:
# SQL query to create main table for email data
dataquery = """  \
select campaignid, campaignname, subjectline, CampaignCategory, startdate, enddate, SUM(sendcount) as sendcount, SUM(receivedcount) as receivedcount, SUM(uniqueopenscount) as uniqueopenscount
             , SUM(uniqueclickscount) as uniqueclickscount, SUM(retailsalesamt) as retailsalesamount
             , SUM(totalsalesamt) as totalsalesamount
from fact_CheetahCampMetricsbyDay cm
inner join dim_Date dd on cm.startdate = dd.SQLDateStamp
where dd.FY_No = 2019
and dd.FP_No = 7
and CampaignCategory in ('Daily Marketing','Control','Segmented Daily Marketing')
group by campaignid, campaignname, subjectline, CampaignCategory,startdate, enddate 
"""

# Creation of email Table
em_table = pd.read_sql_query(dataquery,dw)

# Separate Emails into Segments
em_table['Segment'] = ["Art" if "ART_ART" in x  else "Scrapbooking" if "SCRAPBOOKING_SCRAPBOOKING" in x 
                       else "Yarn" if "YARN_YARN" in x else "General" for x in em_table['campaignname']]

# Calculate Open Rate
em_table['openrate'] = em_table.uniqueopenscount / em_table.receivedcount

# Sort and Rank by Segment and Open Rate
em_table = em_table.sort_values(by=['Segment','openrate'], ascending=[True,False])
em_table['rank_open'] = em_table.groupby('Segment')['openrate'].rank(ascending=False)

# Sort and Rank by Segment and Total Sales
em_table = em_table.sort_values(by=['Segment','totalsalesamount'], ascending=[True,False])
em_table['rank_sales'] = em_table.groupby('Segment')['totalsalesamount'].rank(ascending=False)

# Clean Up the Campaign Name
em_table['campaignname'] = em_table['campaignname'].str.split('\(2019').str[0]

# Create Abridged Campaign Name
em_table['campabr'] = em_table['campaignname'].str.split('_').str[0] + " " + em_table['campaignname'].str.split('_').str[3]

#Calculate Sales Per Customer
em_table['salesper'] = em_table.totalsalesamount / em_table.receivedcount

#output the EM summary table by campaign
em_table.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\emtable.xlsx')

em_table.head(10)

,campaignid,campaignname,subjectline,CampaignCategory,startdate,enddate,sendcount,receivedcount,uniqueopenscount,uniqueclickscount,retailsalesamount,totalsalesamount,Segment,openrate,rank_open,rank_sales,campabr,salesper
36,12843,07.08 Monday Segments (First)_ART_ART_AUDIENCE,Your Special CANVAS Coupon,Segmented Daily Marketing,2019-07-08,2019-07-08,80777.0,80711.0,13829.0,895.0,11661.4850,11661.4850,Art,0.171340,2.0,1.0,07.08 Monday Segments (First) AUDIENCE,0.144484
57,12978,07.12 Friday Segments (First)_ART_ART_AUDIENCE,We are Bringing the ðŸ”¥ðŸ”¥,Segmented Daily Marketing,2019-07-12,2019-07-12,81041.0,80972.0,11741.0,751.0,11316.3350,11316.3350,Art,0.145001,21.0,2.0,07.12 Friday Segments (First) AUDIENCE,0.139756
5,12668,07.01 Monday Segments (First)_ART_ART_AUDIENCE,It's World Watercolor Month!,Segmented Daily Marketing,2019-07-01,2019-07-01,80428.0,80261.0,13074.0,1025.0,9604.0500,9604.0500,Art,0.162894,11.0,3.0,07.01 Monday Segments (First) AUDIENCE,0.119660
120,13412,07.26 Friday First (Segments)_ART_ART_AUDIENCE,Your TOTAL PURCHASE Coupon is ðŸ”¥ðŸ”¥,Segmented Daily Marketing,2019-07-26,2019-07-26,79387.0,79303.0,12457.0,651.0,8968.9800,8968.9800,Art,0.157081,12.0,4.0,07.26 Friday First (Segments) AUDIENCE,0.113098
101,13249,07.22 Monday Segments (First)_ART_ART_AUDIENCE,Stock Up on Supplies + Get $10 Off,Segmented Daily Marketing,2019-07-22,2019-07-22,79807.0,79716.0,12412.0,614.0,8179.8500,8179.8500,Art,0.155703,13.0,5.0,07.22 Monday Segments (First) AUDIENCE,0.102612
46,12918,07.10 Wednesday Segments_ART_ART_AUDIENCE,These Deals are a Perfect 10 ðŸ’µ,Segmented Daily Marketing,2019-07-10,2019-07-10,80682.0,80415.0,12434.0,830.0,7731.2133,7731.2133,Art,0.154623,14.0,6.0,07.10 Wednesday Segments AUDIENCE,0.096141
25,12781,07.05 Friday Segments (First)_ART_ART_AUDIENCE,You Can Still Save on Canvas!,Segmented Daily Marketing,2019-07-05,2019-07-05,80011.0,79904.0,10953.0,672.0,7652.1136,7652.1136,Art,0.137077,23.0,7.0,07.05 Friday Segments (First) AUDIENCE,0.095766
79,13145,07.17 Wednesday Segments (First)_ART_ART_AUDIE...,We Predict You Will Save $10 ðŸ”®,Segmented Daily Marketing,2019-07-17,2019-07-17,80066.0,79994.0,12249.0,414.0,6970.9150,6970.9150,Art,0.153124,15.0,8.0,07.17 Wednesday Segments (First) AUDIENCE,0.087143
14,12732,07.03 Wednesday Segments (First)_ART_ART_AUDIE...,BOGO Free Canvas Ends Today!,Segmented Daily Marketing,2019-07-03,2019-07-03,80119.0,79821.0,11869.0,560.0,6926.4950,6926.4950,Art,0.148695,19.0,9.0,07.03 Wednesday Segments (First) AUDIENCE,0.086775
111,13371,07.24 Wednesday First (Segments)_ART_ART_AUDIE...,Save BIG on Your Next Project,Segmented Daily Marketing,2019-07-24,2019-07-24,79578.0,79440.0,11941.0,524.0,6720.6600,6720.6600,Art,0.150315,18.0,10.0,07.24 Wednesday First (Segments) AUDIENCE,0.084600


In [178]:
# subquery to get rollup metrics by segment

sub = """ \
select segment, SUM(sendcount) as sendcount, SUM(receivedcount) as receivedcount, SUM(uniqueopenscount) as uniqueopenscount
             , SUM(uniqueclickscount) as uniqueclickscount
             , SUM(retailsalesamount) as retailsalesamount
             , SUM(totalsalesamount) as totalsalesamount 
from em_table
group by segment
"""

rollup=ps.sqldf(sub,locals())

rollup['retailsalesamount'] = rollup['retailsalesamount'].astype('int64')
rollup['totalsalesamount'] = rollup['totalsalesamount'].astype('int64')

rollup

,Segment,sendcount,receivedcount,uniqueopenscount,uniqueclickscount,retailsalesamount,totalsalesamount
0,Art,1010272.0,1008693.0,154088.0,8344.0,103983,103983
1,General,58790072.0,58591101.0,9093909.0,412615.0,3151985,3151985
2,Scrapbooking,702276.0,701146.0,122552.0,8739.0,92407,92407
3,Yarn,765973.0,764765.0,178082.0,15646.0,136948,136948


In [181]:
# isolate Segment and Control campaigns for incremental calculation and combine dfs

em_cont = em_table[em_table['CampaignCategory'] == 'Control']
em_seg = em_table[em_table['CampaignCategory'] == 'Segmented Daily Marketing']
em_test = [em_seg, em_cont]
em_inc = pd.concat(em_test)

# Remove any "General" campaigns, as there is no corresponding control
em_inc = em_inc[em_inc.Segment != 'General']



# Sort table and calculate incremental (current sales per customer - prior) * received 
# Then delete the control rows
em_srt = em_inc.sort_values(by=['campaignname','CampaignCategory'], ascending=[False,True])
em_srt['incsalesper'] = em_srt.salesper.diff()
em_srt['incsales'] = em_srt.incsalesper * em_srt.receivedcount
em_res = em_srt[em_srt.CampaignCategory != 'Control']

In [182]:
#Rollup incremental by segment and merge to Rollup

subi = """ \
select segment, sum(incsales) as incsales
from em_res
group by segment
"""

incroll=ps.sqldf(subi,locals())
rollup= pd.merge(rollup,incroll, how='left', on='Segment')



# Create table from em_inc to output for ranks for Revenue and Opens

subr = """ \
select et.segment, rank_open, openrate, rank_sales, totalsalesamount, subjectline, campaignname, campabr, maxrank
from em_table et
inner join (select segment, max(rank_open) as maxrank from em_table group by segment) r on et.segment = r.segment
order by et.segment, rank_open, openrate, rank_sales, totalsalesamount, subjectline, campaignname
"""

rankdata=ps.sqldf(subr,locals())

In [184]:
# Output tables


rollup.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\summary.xlsx')
em_inc.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\campsummary.xlsx')
em_res.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\incremental.xlsx')
rankdata.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\rank.xlsx')

In [146]:
# SQL query to create response table for email data
respquery = """  \
select (case when campaignname like '%ART_ART%' then 'Art'
      else case when campaignname like '%SCRAPBOOKING_SCRAPBOOKING%' then 'Scrapbooking'
      else case when campaignname like '%YARN_YARN%' then 'Yarn'
      else 'General' end end end) as segment
      , count(distinct emailaddress) as deliv
      , count(distinct case when totalsalesamtsplit > 0 then emailaddress end) as responders

from fact_CheetahCampMetricsLayer cm
inner join dim_Date dd on cm.startdate = dd.SQLDateStamp
where dd.FY_No = 2019
and dd.FP_No = 7
and CampaignCategory in ('Daily Marketing','Control','Segmented Daily Marketing')
and cm.IsDelievered = 1
group by (case when campaignname like '%ART_ART%' then 'Art'
      else case when campaignname like '%SCRAPBOOKING_SCRAPBOOKING%' then 'Scrapbooking'
      else case when campaignname like '%YARN_YARN%' then 'Yarn'
      else 'General' end end end)
order by (case when campaignname like '%ART_ART%' then 'Art'
      else case when campaignname like '%SCRAPBOOKING_SCRAPBOOKING%' then 'Scrapbooking'
      else case when campaignname like '%YARN_YARN%' then 'Yarn'
      else 'General' end end end) 
"""

# Creation of Response Table
resp_table = pd.read_sql_query(respquery,dw)

In [149]:
resp_table['resprate'] = resp_table.responders / resp_table.deliv
resp_table.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\resp_table.xlsx')

In [210]:
# SQL query to create response table for email data
linkquery = """  \
select (case when camp_name like '%ART_ART%' then 'Art'
      else case when camp_name like '%SCRAPBOOKING_SCRAPBOOKING%' then 'Scrapbooking'
      else case when camp_name like '%YARN_YARN%' then 'Yarn'
      else 'General' end end end) as segment, Link_Name, COUNT(distinct id) as clicks
from fact_CheetahFeedback cm
inner join dim_Date dd on cm.startdate = dd.SQLDateStamp
where dd.FY_No = 2019
and dd.FP_No = 7
and FeedbackSourceCode = 3
and CampaignType in ('Daily Marketing','Control','Segmented Daily Marketing')
group by (case when camp_name like '%ART_ART%' then 'Art'
      else case when camp_name like '%SCRAPBOOKING_SCRAPBOOKING%' then 'Scrapbooking'
      else case when camp_name like '%YARN_YARN%' then 'Yarn'
      else 'General' end end end), Link_Name
order by  (case when camp_name like '%ART_ART%' then 'Art'
      else case when camp_name like '%SCRAPBOOKING_SCRAPBOOKING%' then 'Scrapbooking'
      else case when camp_name like '%YARN_YARN%' then 'Yarn'
      else 'General' end end end) , Link_Name
"""

# Creation of Response Table
link_table = pd.read_sql_query(linkquery,dw)

In [223]:
# Clean Up the link Name
link_table['link_abr'] = link_table['Link_Name'].str.split('- ').str[1]
link_table['link_abr'] = link_table['link_abr'].str.split('?').str[0]
link_table['link_abr'] = pd.np.where(link_table.link_abr.str.contains("weekly-ad"),"/weekly-ad",link_table.link_abr)
link_table['link_abr'] = pd.np.where(link_table.link_abr.str.contains("product"),"/product",link_table.link_abr)
link_table['link_abr'] = pd.np.where(link_table.link_abr.str.contains("projects"),"/projects",link_table.link_abr)
link_table['link_abr'] = pd.np.where(link_table.link_abr.str.contains("official"),"/officialacmoore",link_table.link_abr)
link_table['link_abr'] = pd.np.where(link_table.link_abr.str.contains("maps"),"/maps",link_table.link_abr)
link_table['link_abr'] = pd.np.where(link_table.link_abr.str.contains("SundayCoupons"),"/SundayCoupons",link_table.link_abr)




# Create table from em_inc to output for ranks for Revenue and Opens

subr = """ \
select segment, link_abr, sum(clicks) as clicks
from link_table
group by segment, link_abr
order by segment, link_abr
"""

linkdata=ps.sqldf(subr,locals())

# Sort and Rank by Segment and Clicks
linkdata = linkdata.sort_values(by=['segment','clicks'], ascending=[True,False])
linkdata['rank_clicks'] = linkdata.groupby('segment')['clicks'].rank(ascending=False, method='first')


subr = """ \
select lt.segment, rank_clicks, link_abr, clicks, maxrank, tot_clicks
from linkdata lt
inner join (select segment, max(rank_clicks) as maxrank, sum(clicks) as tot_clicks from linkdata group by segment) aa on lt.segment = aa.segment

"""

linkdata=ps.sqldf(subr,locals())

linkdata['pct'] = linkdata.clicks / linkdata.tot_clicks

link_table.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\click_table.xlsx')
linkdata.to_excel('Q:\\Confidential\\AC Moore\\Analytics\\Analysis\\2019\\201906 - Monthly Email Analysis\\click_data.xlsx')